In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [2]:
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet

In [3]:
def test_intel_tensorflow() :
    import tensorflow as tf
    print("We are using Tensorflow version {}".format(tf.__version__))
    
    major_version = int(tf.__version__.split(".")[0])
    if major_version >= 2 :
        from tensorflow.python.util import _pywrap_util_port
        print("Intel-optimizations (DNNL) enabled:", _pywrap_util_port.IsMklEnabled())
    else :
        print("Intel-optimizations  (DNNL) enabled:")

test_intel_tensorflow()

We are using Tensorflow version 2.8.4
Intel-optimizations (DNNL) enabled: False


In [4]:
fx.init('keras_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── .workspace
├── agg_to_col_one_signed_cert.zip
├── agg_to_col_two_signed_cert.zip
├── cert
├── data
├── final_model
│   ├── assets
│   ├── keras_metadata.pb
│   ├── saved_model.pb
│   └── variables
│       ├── variables.data-00000-of-00001
│       └── variables.index
├── logs
│   └── tensorboard
│       ├── events.out.tfevents.1681111804.DESKTOP-8R9A5NI
│       └── events.out.tfevents.1681342139.DESKTOP-8R9A5NI
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   └── plan.yaml
├── requirements.txt
├── save
│   ├── keras_cnn_mnist_best.pbuf
│   ├── keras_cnn_mnist_init.pbuf
│   └── keras_cnn_mnist_last.pbuf
└── src
    ├── keras_cnn.py
    ├── mnist_utils.py
    ├── tfmnist_inmemory.py
    └── __init__.py

10 directories, 21 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2

In [5]:
#Import and process training, validation, and test images/labels

# Set the ratio of validation imgs, can't be 0.0
VALID_PERCENT = 0.3

(X_train, y_train), (X_test, y_test) = mnist.load_data()
split_on = int((1 - VALID_PERCENT) * len(X_train))

train_images = X_train[0:split_on,:,:]
train_labels = to_categorical(y_train)[0:split_on,:]

valid_images = X_train[split_on:,:,:]
valid_labels = to_categorical(y_train)[split_on:,:]

test_images = X_test
test_labels = to_categorical(y_test)

In [6]:
def preprocess(images):
    #Normalize
    images = (images / 255) - 0.5
    #Flatten
    images = images.reshape((-1, 784))
    return images

# Preprocess the images.
train_images = preprocess(train_images)
valid_images = preprocess(valid_images)
test_images = preprocess(test_images)

feature_shape = train_images.shape[1]
classes = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

def build_model(feature_shape,classes):
    #Defines the MNIST model
    model = Sequential()
    model.add(Dense(64, input_shape=feature_shape, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)
    return model

In [7]:
#Create a federated model using the build model function and dataset
fl_model = FederatedModel(build_model,data_loader=fl_data)

In [8]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [9]:
#Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/keras_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/keras_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/keras_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": true,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups.0.name": "train_and_validate",
    "assigner.settings.task_groups.0.percentage": 1.0,
    "assigner.settings.task_groups.0.tasks.0": "aggregated_model_validation",
    "assigner.settings.task_groups.0.tasks.1": "train",
    "assigner.settings.task_groups.0.tasks.2": "locally_tuned_model_validation",
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.t

In [10]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,override_config={'aggregator.settings.rounds_to_train':5})

[10:46:23] INFO     Updating aggregator.settings.rounds_to_train to 5...                                                                           ]8;id=512324;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=754437;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#102\102]8;;\

           INFO     FL-Plan hash is 3038ce92442cac7666b80fc10e51468e5e5adcc0b61540dc858ee7d85c8b0c885c2cc942842464f636d17c74cb36ced1                 ]8;id=94423;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=77420;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py#235\235]8;;\

           INFO     Building 🡆 Object NoCompressionPipeline from openfl.pipelines Module.                                                            ]8;id=774210;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=152580;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py#171\171]8;;\

           INFO     Creating Initial Weights File    🠆 save/keras_cnn_mnist_init.pbuf                                                              ]8;id=280210;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=999989;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#277\277]8;;\

           INFO     Starting Experiment...                                                                                                         ]8;id=564537;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=911394;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#281\281]8;;\

...

           METRIC   Round 4: saved the best model with score 0.952722                                                                          ]8;id=377310;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=554699;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#858\858]8;;\

           METRIC   Round 4, aggregator: locally_tuned_model_validation                                                                        ]8;id=452388;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=423353;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#842\842]8;;\
                    <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at 0x000002302EDD6A60> accuracy:                            
                    0.950000                                                                                                                                    

           INFO     Saving round 5 model...                                                                                                    ]8;id=268173;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=261008;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#894\894]8;;\

           INFO     Experiment Completed. Cleaning up...                                                                                       ]8;id=218224;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=19963;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#899\899]8;;\

           INFO     All tasks completed on two for round 4...                                                                                ]8;id=869273;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\collaborator\collaborator.py\collaborator.py]8;;\:]8;id=929598;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\collaborator\collaborator.py#171\171]8;;\

In [11]:
#Save final model and load into keras
final_fl_model.save_native('final_model')
model = tf.keras.models.load_model('./final_model')

In [12]:
#Test the final model on our test set
model.evaluate(test_images,test_labels)

313/313 [==============================] - 0s 516us/step - loss: 0.1358 - accuracy: 0.9566


[0.1357623189687729, 0.95660001039505]

In [13]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [14]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 42000
Original validation data size: 18000

Collaborator one's training data size: 21000
Collaborator one's validation data size: 9000

Collaborator two's training data size: 21000
Collaborator two's validation data size: 9000



In [15]:
#Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/keras_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/keras_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/keras_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": true,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups.0.name": "train_and_validate",
    "assigner.settings.task_groups.0.percentage": 1.0,
    "assigner.settings.task_groups.0.tasks.0": "aggregated_model_validation",
    "assigner.settings.task_groups.0.tasks.1": "train",
    "assigner.settings.task_groups.0.tasks.2": "locally_tuned_model_validation",
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.t

In [ ]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,override_config={'aggregator.settings.rounds_to_train':5})

[10:46:45] INFO     Updating aggregator.settings.rounds_to_train to 5...                                                                           ]8;id=84220;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=890661;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#102\102]8;;\

           INFO     FL-Plan hash is 3038ce92442cac7666b80fc10e51468e5e5adcc0b61540dc858ee7d85c8b0c885c2cc942842464f636d17c74cb36ced1                 ]8;id=882670;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=866945;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py#235\235]8;;\

           INFO     Building 🡆 Object NoCompressionPipeline from openfl.pipelines Module.                                                            ]8;id=533242;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=592893;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\plan\plan.py#171\171]8;;\

           INFO     Creating Initial Weights File    🠆 save/keras_cnn_mnist_init.pbuf                                                              ]8;id=196590;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=493131;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#277\277]8;;\

           INFO     Starting Experiment...                                                                                                         ]8;id=445073;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py\native.py]8;;\:]8;id=353553;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\native\native.py#281\281]8;;\

...

           INFO     Collaborator two is sending task results for aggregated_model_validation, round 1                                          ]8;id=720380;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=66001;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#520\520]8;;\

           METRIC   Round 1, collaborator validate_agg aggregated_model_validation result accuracy: 0.891000                                   ]8;id=972778;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py\aggregator.py]8;;\:]8;id=541806;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\aggregator\aggregator.py#559\559]8;;\

           INFO     Using TaskRunner subclassing API                                                                                         ]8;id=40268;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\collaborator\collaborator.py\collaborator.py]8;;\:]8;id=255318;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\component\collaborator\collaborator.py#253\253]8;;\

           INFO     Run 0 epoch of 1 round                                                                                                    ]8;id=841938;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\task\runner_keras.py\runner_keras.py]8;;\:]8;id=112441;file://C:\Users\BHSU\AppData\Local\Programs\Python\Python39\lib\site-packages\openfl\federated\task\runner_keras.py#83\83]8;;\

     73/Unknown - 0s 694us/step - loss: 0.3519 - accuracy: 0.8968

In [ ]:
#Save final model and load into keras
final_fl_model.save_native('final_model')
model = tf.keras.models.load_model('./final_model')

In [ ]:
#Test the final model on our test set
model.evaluate(test_images,test_labels)